In [3]:
import pandas as pd

taxi_zones_socioeconomics = pd.read_csv("Data/taxi_zones_with_socioeconomics.csv")

taxi_zones_socioeconomics.head()

,LocationID,Zone,medincome,total_pop1,fpl_100,fpl_100to150,median_rent,total_hholds1,hholds_snap,over16total_industry1,...,age65plus_noinsurance,hisplat_raceethnic,nonhispLat_white_raceethnic,nonhispLat_black_raceethnic,nonhispLat_amerindian_raceethnic,nonhispLat_asian_raceethnic,age65_plus,fpl_150,not_insured,ZCTA_IDs
0,1.0,Newark Airport,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
1,2.0,Jamaica Bay,NaN,14.393144,2.066430,1.199864,NaN,5.336487,NaN,6.874121,...,0.032127,0.041152,0.050651,0.035635,0.000000,0.005637,0.292471,0.046583,0.009594,"11430, 11693"
2,3.0,Allerton/Pelham Gardens,65397.817752,72380.218166,10893.432712,5640.781364,1412.999993,22298.427261,19.689440,32802.383705,...,173.980671,28.973197,10.043446,52.842447,0.199988,6.120936,1665.881687,22.840158,5.720188,"10469, 10475, 10461"
3,4.0,Alphabet City,63467.357932,58144.201429,13676.276449,3518.376196,1716.210060,29679.076454,18.046047,31717.662880,...,67.009909,23.721156,51.352437,7.785093,0.070841,14.080305,1391.336791,29.472190,4.696983,"10009, 10002"
4,5.0,Arden Heights,97407.000000,61392.000000,3602.000000,2937.000000,1280.000000,21318.000000,3.730000,28106.000000,...,151.000000,11.340000,81.180000,0.840000,0.150000,5.320000,1934.000000,10.650000,2.130000,10312


### create clusters using k-means first focussing on 2 variables: median income and tree entropy (as proxy for)

In [24]:
taxizones = pd.read_csv("Data/taxi_zones_geometry.csv")
parks = pd.read_csv("Data/Heat_Vulnerability/Parks_Properties_20231208.csv")

In [18]:
taxizones

,Unnamed: 0,borough,location_i,objectid,shape_area,shape_leng,zone,geometry
0,0,EWR,1.0,1.0,0.000782,0.116357,Newark Airport,POLYGON ((-74.18445299999996 40.69499599999990...
1,1,Queens,2.0,2.0,0.004866,0.433470,Jamaica Bay,MULTIPOLYGON (((-73.82337597260663 40.63898704...
2,2,Bronx,3.0,3.0,0.000314,0.084341,Allerton/Pelham Gardens,POLYGON ((-73.84792614099985 40.87134223399991...
3,3,Manhattan,4.0,4.0,0.000112,0.043567,Alphabet City,POLYGON ((-73.97177410965318 40.72582128133706...
4,4,Staten Island,5.0,5.0,0.000498,0.092146,Arden Heights,"POLYGON ((-74.17421738099989 40.5625680859999,..."
...,...,...,...,...,...,...,...,...
258,258,Brooklyn,256.0,256.0,0.000169,0.067915,Williamsburg (South Side),POLYGON ((-73.95834207500002 40.71330630099992...
259,259,Bronx,259.0,259.0,0.000395,0.126750,Woodlawn/Wakefield,POLYGON ((-73.85107116191898 40.91037152011096...
260,260,Queens,260.0,260.0,0.000422,0.133514,Woodside,POLYGON ((-73.90175373399988 40.76077547499996...
261,261,Manhattan,261.0,261.0,0.000034,0.027120,World Trade Center,"POLYGON ((-74.01332610899988 40.7050307879999,..."


In [9]:
parks

,ACQUISITIONDATE,ACRES,ADDRESS,BOROUGH,CLASS,COMMUNITYBOARD,COUNCILDISTRICT,DEPARTMENT,EAPPLY,GISOBJID,...,PRECINCT,RETIRED,SIGNNAME,SUBCATEGORY,TYPECATEGORY,URL,US_CONGRESS,WATERFRONT,ZIPCODE,multipolygon
0,1937-06-01 00:00:00.0000000,249.389,NaN,Q,PARK,401403404406,1921222429,Q-03,Grand Central Parkway Ext,100000441.0,...,114,False,Grand Central Parkway Extension,EXWY,Parkway,http://www.nycgovparks.org/parks/Q084A/,614.0,True,11103113671136811369113701137111375,MULTIPOLYGON (((-73.8587476480729 40.767414466...
1,2009-11-20 00:00:00.0000000,9.375,NaN,Q,PARK,414,32,Q-14,Sunset Cove Park,100003992.0,...,100,False,Sunset Cove Park,NaN,Undeveloped,http://www.nycgovparks.org/parks/Q498/,5.0,True,11693,MULTIPOLYGON (((-73.82218300936414 40.59892072...
2,1949-03-24 00:00:00.0000000,2.035,61-25 LITTLE NECK PARKWAY,Q,PARK,411,23,Q-11,Challenge Playground,100000009.0,...,111,False,Challenge Playground,JOP,Jointly Operated Playground,http://www.nycgovparks.org/parks/Q346/,3.0,False,11362,MULTIPOLYGON (((-73.72738293199147 40.75605209...
3,NaN,0.356,200-15 MCLAUGHLIN AV,Q,PARK,408,23,Q-08,McLaughlin Playground,100000044.0,...,107,False,McLaughlin Playground,Neighborhood Plgd,Playground,http://www.nycgovparks.org/parks/Q021B/,5.0,False,11423,MULTIPOLYGON (((-73.76598994603171 40.72549572...
4,1938-06-24 00:00:00.0000000,2.948,73-80 260 STREET,Q,PARK,413,23,Q-13,Glen Oaks Oval /Tenney Park,100000138.0,...,105,False,Glen Oaks Oval,Neighborhood Plgd,Community Park,http://www.nycgovparks.org/parks/Q310/,3.0,False,11004,MULTIPOLYGON (((-73.71534734971091 40.74896585...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040,NaN,0.800,NaN,B,PARK,314,45,B-14,Flatbush Malls,100005051.0,...,70,False,Flatbush Malls,Sitting Area/Triangle/Mall,Mall,http://www.nycgovparks.org/parks/B179/,9.0,False,11230,MULTIPOLYGON (((-73.96116055481573 40.63413885...
2041,1938-03-08 00:00:00.0000000,0.313,NaN,X,PARK,201,8,X-01,Gouverneur Morris Triangle,100004413.0,...,40,False,Gouverneur Morris Triangle,Sitting Area/Triangle/Mall,Triangle/Plaza,http://www.nycgovparks.org/parks/X139/,15.0,False,10454,MULTIPOLYGON (((-73.91284574336954 40.80461244...
2042,1938-01-01 00:00:00.0000000,0.047,NaN,M,PARK,102,1,M-02,Lt. Joseph Petrosino Square,100004585.0,...,5,False,Petrosino Square,Sitting Area/Triangle/Mall,Triangle/Plaza,http://www.nycgovparks.org/parks/M170/,10.0,False,10012,MULTIPOLYGON (((-73.9974421394935 40.721593227...
2043,1908-04-09 00:00:00.0000000,1.300,NaN,M,PARK,109,7,M-09,Broadway Malls 110th-122nd,100005116.0,...,26,False,Broadway Malls,Sitting Area/Triangle/Mall,Mall,http://www.nycgovparks.org/parks/M094A/,NaN,False,1002510027,MULTIPOLYGON (((-73.9638091050991 40.808084800...


In [25]:
import geopandas as gpd
from shapely import wkt

# Assuming 'taxizones' and 'parks' are your initial DataFrames

epsg_code = 4326


# Convert the 'geometry' column in taxizones and 'multipolygon' in parks to a GeoSeries
taxizones['geometry'] = taxizones['geometry'].apply(wkt.loads)
parks['multipolygon'] = parks['multipolygon'].apply(wkt.loads)

# Convert DataFrames to GeoDataFrames
gdf_taxizones = gpd.GeoDataFrame(taxizones, geometry='geometry')
gdf_parks = gpd.GeoDataFrame(parks, geometry='multipolygon')


# Check and set CRS for taxizones if undefined
if gdf_taxizones.crs is None:
    # Replace 'epsg_code' with the correct EPSG code
    gdf_taxizones.set_crs(epsg=epsg_code, inplace=True)

# Check and set CRS for parks if undefined
if gdf_parks.crs is None:
    # Assuming parks should have the same CRS as taxizones
    gdf_parks.set_crs(gdf_taxizones.crs, inplace=True)


# Ensure both GeoDataFrames use the same CRS
gdf_parks.set_crs(gdf_taxizones.crs, inplace=True)

joined = gpd.sjoin(gdf_taxizones, gdf_parks, how="inner", op='intersects')

# Calculate the intersection area
# 'geometry' refers to the geometry of gdf_taxizones, and 'geometry_parks' to that of gdf_parks
joined['intersection_area'] = joined.apply(
    lambda row: row['geometry'].intersection(row['geometry_parks']).area, axis=1)

# Group by taxi zone and sum the intersection areas
park_area_per_taxizone = joined.groupby('location_i')['intersection_area'].sum()

# Calculate the proportion of park area in each taxi zone
taxizone_area = gdf_taxizones.set_index('location_i')['shape_area']
proportion_park_area = park_area_per_taxizone / taxizone_area

# The result is a series with the proportion of park area for each taxizone
print(proportion_park_area)

""" # Calculate the intersection area
joined['intersection_area'] = joined.apply(lambda row: row['geometry'].intersection(row['multipolygon']).area, axis=1)

# Group by taxi zone and sum the intersection areas
park_area_per_taxizone = joined.groupby('location_i')['intersection_area'].sum()

# Calculate the proportion of park area in each taxi zone
taxizone_area = gdf_taxizones.set_index('location_i')['shape_area']
proportion_park_area = park_area_per_taxizone / taxizone_area

# The result is a series with the proportion of park area for each taxizone
print(proportion_park_area) """


c:\Anaconda\anaconda3\envs\dl_intro\Lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


" # Calculate the intersection area\njoined['intersection_area'] = joined.apply(lambda row: row['geometry'].intersection(row['multipolygon']).area, axis=1)\n\n# Group by taxi zone and sum the intersection areas\npark_area_per_taxizone = joined.groupby('location_i')['intersection_area'].sum()\n\n# Calculate the proportion of park area in each taxi zone\ntaxizone_area = gdf_taxizones.set_index('location_i')['shape_area']\nproportion_park_area = park_area_per_taxizone / taxizone_area\n\n# The result is a series with the proportion of park area for each taxizone\nprint(proportion_park_area) "

In [32]:
joined

,Unnamed: 0,borough,location_i,objectid,shape_area,shape_leng,zone,geometry,index_right,ACQUISITIONDATE,...,PIP_RATABLE,PRECINCT,RETIRED,SIGNNAME,SUBCATEGORY,TYPECATEGORY,URL,US_CONGRESS,WATERFRONT,ZIPCODE
1,1,Queens,2.0,2.0,0.004866,0.433470,Jamaica Bay,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ...",1,2009-11-20 00:00:00.0000000,...,False,100,False,Sunset Cove Park,NaN,Undeveloped,http://www.nycgovparks.org/parks/Q498/,5.0,True,11693
32,32,Queens,30.0,30.0,0.000146,0.094510,Broad Channel,"POLYGON ((-73.82076 40.61523, -73.81769 40.614...",1,2009-11-20 00:00:00.0000000,...,False,100,False,Sunset Cove Park,NaN,Undeveloped,http://www.nycgovparks.org/parks/Q498/,5.0,True,11693
1,1,Queens,2.0,2.0,0.004866,0.433470,Jamaica Bay,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ...",40,1954-01-14 00:00:00.0000000,...,False,113,False,Jamaica Bay Park,Large Park,Nature Area,http://www.nycgovparks.org/parks/Q309/,NaN,True,110961143011691
77,77,Queens,86.0,86.0,0.000623,0.134245,Far Rockaway,"POLYGON ((-73.76669 40.61423, -73.76663 40.614...",40,1954-01-14 00:00:00.0000000,...,False,113,False,Jamaica Bay Park,Large Park,Nature Area,http://www.nycgovparks.org/parks/Q309/,NaN,True,110961143011691
136,136,Queens,132.0,132.0,0.002038,0.245479,JFK Airport,"MULTIPOLYGON (((-73.82503 40.66358, -73.82486 ...",40,1954-01-14 00:00:00.0000000,...,False,113,False,Jamaica Bay Park,Large Park,Nature Area,http://www.nycgovparks.org/parks/Q309/,NaN,True,110961143011691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,260,Queens,260.0,260.0,0.000422,0.133514,Woodside,"POLYGON ((-73.90175 40.76078, -73.90147 40.759...",550,1946-12-20 00:00:00.0000000,...,True,114,False,St. Michael's Playground,Neighborhood Plgd,Neighborhood Park,http://www.nycgovparks.org/parks/Q174A/,14.0,False,11377
262,262,Manhattan,262.0,262.0,0.000122,0.049064,Yorkville East,"MULTIPOLYGON (((-73.94383 40.78286, -73.94376 ...",1091,1953-10-08 00:00:00.0000000,...,False,19,False,Mill Rock Park,Large Park,Nature Area,http://www.nycgovparks.org/parks/M209/,NaN,True,10128
262,262,Manhattan,262.0,262.0,0.000122,0.049064,Yorkville East,"MULTIPOLYGON (((-73.94383 40.78286, -73.94376 ...",1374,1876-04-01 00:00:00.0000000,...,False,19,False,Carl Schurz Park,Large Park,Neighborhood Park,http://www.nycgovparks.org/parks/M081/,12.0,True,10028
262,262,Manhattan,262.0,262.0,0.000122,0.049064,Yorkville East,"MULTIPOLYGON (((-73.94383 40.78286, -73.94376 ...",888,1989-01-18 00:00:00.0000000,...,False,19,False,Asphalt Green,Neighborhood Plgd,Community Park,http://www.nycgovparks.org/parks/M286/,12.0,False,10128


### create sanky plot to calculate the deviation change from mean for each cluster